In [1]:
# Import the necessary libraries

import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from imblearn.over_sampling import SMOTE
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime, date

In [2]:
# Create a connection to the database

path_name = "./engagement.db"
conn = sqlite3.connect(path_name)

In [3]:
# SQL functions

sql = lambda query: pd.read_sql_query(query, conn)
sql_to_clipboard = lambda query: sql(query).to_clipboard(index=False)

In [4]:
# Set coherent theme for visualizations

sns.set_theme(style="darkgrid")
sns.set_context("notebook", font_scale=1.5) 

In [5]:
# Create dataframe of features vs. enrollment

eng_factors = sql(
"""
    SELECT eligibility.*, client_metadata.income_range, 
    client_metadata.hr_engagement, accounts.account_id
    FROM 
    eligibility JOIN client_metadata
    ON eligibility.client = client_metadata.client
    LEFT JOIN accounts 
    ON eligibility.eligibility_id = accounts.eligibility_id;
""")

DatabaseError: Execution failed on sql '
    SELECT eligibility.*, client_metadata.income_range, 
    client_metadata.hr_engagement, accounts.account_id
    FROM 
    eligibility JOIN client_metadata
    ON eligibility.client = client_metadata.client
    LEFT JOIN accounts 
    ON eligibility.eligibility_id = accounts.eligibility_id;
': no such table: eligibility

In [ ]:
# Change enrollment factors dataframe to input & output dataframes of categorical values
# & prepare date dataframes 

eng_factors['account_id'] = eng_factors['account_id'].notnull().astype('int')
eng_factors = eng_factors.rename(columns = {'account_id': 'account'}).set_index('eligibility_id') # Set [null values] = 0, [evidence of account] = 1
edf = eng_factors.drop(columns = ['sex','family_status', 'income_range', 'hr_engagement']) # Date/continuous variables
ef = eng_factors.drop(columns = ['account', 'benefits_start_date', 'date_of_birth']) # Ensure all variables treated as categorical variables
e = eng_factors['account']

In [ ]:
# Determine impact of start time (function)

def start_func(year):
    year = datetime.strptime(year, "%Y-%m-%d").date()
    return year


In [ ]:
# Determine impact of start date (visualization)
edf.benefits_start_date = pd.to_datetime(edf.benefits_start_date).astype('str')
edf['benefits_start_date'] = edf.benefits_start_date.apply(start_func)

fig, ax = plt.subplots(figsize=(12, 12))
ax = sns.lineplot(data=edf, x="benefits_start_date", y="account", hue="client", palette='tab10')
ax.set(ylabel='Average Enrollment', xlabel='Start Date of Benefits', title = 'Enrollment v. Start Date')
plt.show()
fig.savefig('viz/start_date.png')

# => enrollment dependent on client, not start date

In [ ]:
# Determine impact of age (function)

def age_func(born):
    born = datetime.strptime(born, "%Y-%m-%d").date()
    today = date.today()
    return today.year - born.year - ((today.month, 
                                      today.day) < (born.month, 
                                                    born.day))

In [ ]:
# Determine impact of age (visual)

edf.date_of_birth = pd.to_datetime(edf.date_of_birth).astype('str')
edf['age'] = edf.date_of_birth.apply(age_func)


ax = sns.displot(edf, x="age", hue="account", element="step")
ax.set(ylabel='Average Enrollment', xlabel='Age', title = 'Enrollment v. Age')
plt.savefig('viz/age.png')

In [ ]:
# Regression analysis of age (SMOTE as data is imbalanced)

X = edf['age'].to_numpy()
X = X.reshape(-1,1)
y = edf['account']

# SMOTE

oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

# Regression

logistic_regression= LogisticRegression()
logistic_regression.fit(X_train,y_train)
y_pred=logistic_regression.predict(X_test)

confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])
sns.heatmap(confusion_matrix, annot=True)
print('Accuracy: ',metrics.accuracy_score(y_test, y_pred))
plt.savefig('viz/age_regression.png')

In [ ]:
# Analysis of categorical values

# Chi-squared test of all categorical features

le = LabelEncoder()
oe = OrdinalEncoder()
ef.sex = le.fit_transform(ef.sex)
ef.client = le.fit_transform(ef.client)
ef.income_range = le.fit_transform(ef.income_range)
ef.hr_engagement = le.fit_transform(ef.hr_engagement)
chi_scores = chi2(ef,e)

In [ ]:
# Isolate p_values & plot (higher p-value indicates independence from engagement array)

p_values = pd.Series(chi_scores[1],index = ef.columns)
p_values.sort_values(ascending = False , inplace = True)

fig = plt.figure()
ax = p_values.plot.bar(x='Feature', y='p-value')
ax.set(title = 'P-values of Features')
fig.savefig('viz/P-value_X2.png')

In [ ]:
# Plot features with significant p-values (<0.5)

# Plot HR engagement

fig = plt.figure(figsize=(8, 6))
ax = sns.countplot(x="hr_engagement", hue = "account", data=eng_factors)
ax.set(ylabel='# of Users', xlabel='HR Engagement', title = 'Affect of HR Engagement on User Enrollment')
fig.savefig('viz/HR_engagement.png')

In [ ]:
# Plot Client

fig = plt.figure(figsize=(8, 6))
ax = sns.countplot(x="client", hue = "account", data=eng_factors)
ax.set(ylabel='# of Users', xlabel='Client', title = 'User Enrollment Based on Client')
fig.savefig('viz/client.png')

In [ ]:
# Plot Income Range

fig = plt.figure(figsize=(15, 6))
ax = sns.countplot(x="income_range", hue = "account", data=eng_factors)
ax.set(ylabel='# of Users', xlabel='Income Range', title = 'Affect of Income on User Enrollment')
fig.savefig('viz/income_range.png')

In [ ]:
# Feature selection using Chi-squared test

ef = ef.to_numpy()
e = e.to_numpy() 
ef_train, ef_test, e_train, e_test = train_test_split(ef, e, test_size=0.33, random_state=1)

In [ ]:
# Prepare input data

oe.fit(ef_train)
ef_train_enc = oe.transform(ef_train)
ef_test_enc = oe.transform(ef_test)

In [ ]:
# Prepare output data

le.fit(e_train)
e_train_enc = le.transform(e_train)
e_test_enc = le.transform(e_test)

In [ ]:
# Feature selection

fs = SelectKBest(score_func=chi2, k='all')
print('Test', ef_test.shape, e_test.shape)
fs.fit(ef_train_enc, e_train_enc)
ef_train_fs = fs.transform(ef_train)
ef_test_fs = fs.transform(ef_test)

In [ ]:
# Plot feature significance scores

fig = plt.figure(figsize=(8,6))
for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()
fig.savefig('viz/psignificance.png')

In [ ]:
# Determine activity among clients

# Create dataframe of activity data from active users & their respective clients

spec_df = sql(
"""
    SELECT eligibility.client, accounts.account_id
    FROM eligibility JOIN accounts
    ON eligibility.eligibility_id = accounts.eligibility_id;
""")
account_eng = sql(
"""
    SELECT account_id, 
    COUNT(*) AS `activity` 
    FROM recommendations 
    GROUP BY account_id;
"""
)

account_eng.set_index('account_id')
spec_df.set_index('account_id')
ae = pd.merge(account_eng, spec_df, how='inner', on= 'account_id')

# Create dataframe of activity data from active users & their SEF

sf_df = sql(
"""
    SELECT eligibility.date_of_birth, eligibility.sex, eligibility.family_status, accounts.account_id
    FROM eligibility JOIN accounts
    ON eligibility.eligibility_id = accounts.eligibility_id;
""")

act_sf = pd.merge(sf_df, account_eng, how='inner', on='account_id')
act_sf.date_of_birth = pd.to_datetime(act_sf.date_of_birth).astype('str')
act_sf['age'] = act_sf.date_of_birth.apply(age_func)
act_sf

In [ ]:
# Analysis of age & activity

sns.scatterplot(data=act_sf, x="age", y="activity")
sns.catplot(data=act_sf, x="activity", y="age", kind='box')
sns.displot(act_sf, x="activity", hue="sex", element="step")

# => mostly evenly distributed, age/sex/family_status is not a relevant feature

In [ ]:
# Analysis of categorical values -> numerical output (ANOVA)

import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
# User-related variables

model = ols('activity ~ C(sex) + C(family_status) + C(sex):C(family_status)', data=act_sf).fit()
sm.stats.anova_lm(model, typ=2)

In [ ]:
A = ae.loc[ae['client'] == 'Client A']
B = ae.loc[ae['client'] == 'Client B']
C = ae.loc[ae['client'] == 'Client C']
D = ae.loc[ae['client'] == 'Client D']
E = ae.loc[ae['client'] == 'Client E']
F = ae.loc[ae['client'] == 'Client F']


In [ ]:
# Client-related ANOVA

from scipy.stats import f_oneway

f_oneway(A.activity, B.activity, C.activity, D.activity, E.activity, F.activity)

In [ ]:
# Visualize user activity by client

activity = act_sf.activity
act_sf = act_sf.drop(columns = ['date_of_birth', 'activity']).set_index('account_id')

sns.catplot(x="client", y="activity", kind="box", data=ae, palette='flare', aspect = 2)
plt.savefig('viz/activity_client.png')

In [ ]:
# Specialty analysis

# Count # of accepted claims by specialty 
acc = sql(
"""
        SELECT claims.amount, providers.specialty FROM claims LEFT JOIN providers
        ON claims.provider_id = providers.provider_id;
"""
)

In [ ]:
# Dollar amount paid by specialty 

acc_sum = acc.groupby(["specialty"]).amount.sum().reset_index().to_csv('acc_sum.csv')

fig = plt.figure(figsize = (30, 6))
ax = sns.stripplot(x="specialty", y="amount", data=acc)
ax.set(ylabel='Claims Paid ($)', xlabel='Specialty', title = 'Claim Amount by Specialty')
fig.savefig('viz/claim_spec.png')

In [ ]:
# Percent of claims paid by specialty

percent_claims = sql(
"""
    SELECT specialty, 
    COUNT(*) AS percent 
    FROM providers 
    GROUP BY specialty;
"""
)

percent_claims['percent'] = (percent_claims['percent'] / percent_claims['percent'].sum()) * 100
percent_claims
percent_claims.to_csv('per_claims.csv')

